<a href="https://colab.research.google.com/github/miguelvelascop/PFG_Miguel_Velasco_Puig/blob/main/Entrenamiento_Modelos_PGF_DemonicSalmon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy as sc
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt

En este paso se debe elegir el tipo de etiquetas que se quiere usar dependeniendo del preprocesado realizado (primer bloque para etiquetas binarias, segundo bloque para etiquetas originales)

In [ ]:
dataFileName = "Dataset_DemonicSalmon_procesado_etiqBinarias.csv"
selectedAverage = 'binary'
labels = [0,1]

In [ ]:
dataFileName = "Dataset_DemonicSalmon_procesado_etiqOriginales.csv"
selectedAverage = 'micro'
labels = [0,1,2,3,4,5,6,7,8,9,10,11,12]

Cargamos el dataset, depresión y no depresión por separado, que previamente he mergeado, ya que estaban separados por pacientes

In [ ]:
from google.colab import files
depresjon_data = files.upload()



In [ ]:
data=pd.read_csv(dataFileName,  sep=',',  comment='#')


Pasamos los csv a matrices, quitando los índices


In [ ]:
demonicSalmon_origin = pd.read_csv(dataFileName,  sep=',',  comment='#').to_numpy()


Separamos las etiquetas de los datos

In [ ]:
demonicSalmon_data = demonicSalmon_origin[:, :-1]
demonicSalmon_target = demonicSalmon_origin[:, -1]

print(demonicSalmon_data.size)

Oversampling SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=50)
demonicSalmon_data_resampled, demonicSalmon_target_resampled = smote.fit_resample(demonicSalmon_data, demonicSalmon_target)
print(demonicSalmon_data_resampled.size)


Normalización

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(demonicSalmon_data_resampled)
demonicSalmon_data_resampled_rescaled = scaler.transform(demonicSalmon_data_resampled)

Selección de características por filtros


In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selector = SelectKBest(chi2, k=6)

print(demonicSalmon_data_resampled_rescaled.shape)
demonicSalmon_data_resampled_rescaled = selector.fit_transform(demonicSalmon_data_resampled_rescaled, demonicSalmon_target_resampled)
print(selector.get_support(indices=True))
print(selector.scores_)
print(demonicSalmon_data_resampled_rescaled.shape)

Separamos los datos en 20/80 para tener un conjunto de entrenamiento y una de prueba

Separación con oversampling

In [ ]:
from sklearn.model_selection import train_test_split
train_demonicSalmon_x, test_demonicSalmon_x, train_demonicSalmon_y, test_demonicSalmon_y = train_test_split(demonicSalmon_data_resampled_rescaled, demonicSalmon_target_resampled, test_size=0.2, random_state=42)

Probamos los modelos, entrenando (fit) 10 veces los clasificadores para obtener una media

Primero importo los modelos que usaremos

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.gaussian_process import GaussianProcessClassifier as GPC
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.ensemble import AdaBoostClassifier as ABC
from sklearn.naive_bayes import GaussianNB as NB
from sklearn.neural_network import MLPClassifier as MLP

from sklearn.model_selection import cross_val_score

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score

from sklearn.model_selection import GridSearchCV

Selección de parámetros KNN

In [ ]:
parametros = {'n_neighbors':[1, 10]}
knn = KNN()
accuracyKNN = []
precisionKNN = []
recallKNN = []
f1KNN = []
knn = GridSearchCV(knn, parametros)
for i in range(10):
  knn.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultKNN = knn.predict(test_demonicSalmon_x)
  accuracyKNN.append(accuracy_score(resultKNN, test_demonicSalmon_y))
  precisionKNN.append(precision_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))
  recallKNN.append(recall_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))
  f1KNN.append(f1_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))

Entrenamiento KNN

In [ ]:
resultKNN = knn.fit(train_demonicSalmon_x, train_demonicSalmon_y)


Validación KNN

In [ ]:
scoresKNN = cross_val_score(resultKNN, train_demonicSalmon_x, train_demonicSalmon_y, cv=10)
print(scoresKNN)

Matriz de confusión

In [ ]:
y_pred = knn.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Clacular média y desviación de KNN

In [ ]:
meanAcKNN = np.median(accuracyKNN)
stdAcKNN=np.std(accuracyKNN)
meanPrKNN = np.median(precisionKNN)
stdPrKNN=np.std(precisionKNN)
meanReKNN = np.median(recallKNN)
stdReKNN=np.std(recallKNN)
meanF1KNN = np.median(f1KNN)
stdF1KNN=np.std(f1KNN)

Selección de parámetros SVC

In [ ]:
parametros = {'kernel': ('linear', 'rbf'),'C': [1, 10, 100]}
svm = SVC()
accuracySVC = []
precisionSVC = []
recallSVC = []
f1SVC = []
svm = GridSearchCV(svm, parametros)
for i in range(10):
  svm.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultSVC = svm.predict(test_demonicSalmon_x)
  accuracySVC.append(accuracy_score(resultSVC, test_demonicSalmon_y))
  precisionSVC.append(precision_score(resultSVC, test_demonicSalmon_y,average=selectedAverage))
  recallSVC.append(recall_score(resultSVC, test_demonicSalmon_y,average=selectedAverage))
  f1SVC.append(f1_score(resultSVC, test_demonicSalmon_y,average=selectedAverage))

Entrenamiento SVM

In [ ]:
resultSVM = svm.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación SVM

In [ ]:
scoresSVM = cross_val_score(resultSVM, train_demonicSalmon_x, train_demonicSalmon_y, cv=10)
print(scoresSVM)

In [ ]:
y_pred = svm.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de SVM

In [ ]:
meanAcSVC = np.median(accuracySVC)
stdAcSVC=np.std(accuracySVC)
meanPrSVC = np.median(precisionSVC)
stdPrSVC=np.std(precisionSVC)
meanReSVC = np.median(recallSVC)
stdReSVC=np.std(recallSVC)
meanF1SVC = np.median(f1SVC)
stdF1SVC=np.std(f1SVC)

Selección de parámetros linearSVM

In [ ]:
parametros = {'loss':('hinge','square_hinge'),'intercept_scaling':[1, 10]}
linearSVM = LinearSVC()
accuracyLinearSVM = []
precisionLinearSVM = []
recallLinearSVM = []
f1LinearSVM = []
linearSVM = GridSearchCV(linearSVM, parametros)
for i in range(10):
  linearSVM.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultLinearSVM = linearSVM.predict(test_demonicSalmon_x)
  accuracyLinearSVM.append(accuracy_score(resultLinearSVM, test_demonicSalmon_y))
  precisionLinearSVM.append(precision_score(resultLinearSVM, test_demonicSalmon_y,average=selectedAverage))
  recallLinearSVM.append(recall_score(resultLinearSVM, test_demonicSalmon_y,average=selectedAverage))
  f1LinearSVM.append(f1_score(resultLinearSVM, test_demonicSalmon_y,average=selectedAverage))

Entrenamiento LinearSVM

In [ ]:
modelSVM = linearSVM.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación LinearSVC

In [ ]:
scoresLinearSVM = cross_val_score(modelSVM, train_demonicSalmon_x, train_demonicSalmon_y, cv=10)
print(scoresLinearSVM)

In [ ]:
y_pred = linearSVM.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

In [ ]:
meanAcLinearSVM = np.median(accuracyLinearSVM)
stdAcLinearSVM=np.std(accuracyLinearSVM)
meanPrLinearSVM = np.median(precisionLinearSVM)
stdPrLinearSVM=np.std(precisionLinearSVM)
meanReLinearSVM = np.median(recallLinearSVM)
stdReLinearSVM=np.std(recallLinearSVM)
meanF1LinearSVM = np.median(f1LinearSVM)
stdF1LinearSVM=np.std(f1LinearSVM)
print(meanAcLinearSVM, stdAcLinearSVM,meanPrLinearSVM,stdPrLinearSVM,meanReLinearSVM,stdReLinearSVM,meanF1LinearSVM,stdF1LinearSVM)

Selección de parámetros GPC

In [ ]:
parametros = {'optimizer':('fmin_l_bfgs_b','callable','None')}
gpc = GPC()
accuracyGPC = []
precisionGPC = []
recallGPC = []
f1GPC = []
gpc = GridSearchCV(gpc, parametros)
for i in range(10):
  gpc.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultGPC = gpc.predict(test_demonicSalmon_x)
  accuracyGPC.append(accuracy_score(resultGPC, test_demonicSalmon_y))
  precisionGPC.append(precision_score(resultGPC, test_demonicSalmon_y, average=selectedAverage))
  recallGPC.append(recall_score(resultGPC, test_demonicSalmon_y, average=selectedAverage))
  f1GPC.append(f1_score(resultGPC, test_demonicSalmon_y, average=selectedAverage))


Entrenamiento GPC

In [ ]:
modelGPC = gpc.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación GPC

In [ ]:
scoresGPC = cross_val_score(modelGPC, train_demonicSalmon_x, train_demonicSalmon_y, cv=10)
print(scoresGPC)

Matriz de Confusión

In [ ]:
y_pred = gpc.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de GPC

In [ ]:
meanAcGPC = np.median(accuracyGPC)
stdAcGPC=np.std(accuracyGPC)
meanPrGPC = np.median(precisionGPC)
stdPrGPC=np.std(precisionGPC)
meanReGPC = np.median(recallGPC)
stdReGPC=np.std(recallGPC)
meanF1GPC = np.median(f1GPC)
stdF1GPC=np.std(f1GPC)

Selección de parámetros DTC

In [ ]:
parametros = {'min_samples_split':[1,100],'splitter':('best','random'),'max_features':('auto','sqrt')}
accuracyDTC = []
precisionDTC = []
recallDTC = []
f1DTC = []
dtc = DTC()
dtc = GridSearchCV(dtc, parametros)
for i in range(10):
  dtc.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultDTC = dtc.predict(test_demonicSalmon_x)
  accuracyDTC.append(accuracy_score(resultDTC, test_demonicSalmon_y))
  precisionDTC.append(precision_score(resultDTC, test_demonicSalmon_y, average=selectedAverage))
  recallDTC.append(recall_score(resultDTC, test_demonicSalmon_y, average=selectedAverage))
  f1DTC.append(f1_score(resultDTC, test_demonicSalmon_y, average=selectedAverage))

Entrenamiento DTC

In [ ]:
resultDTC = dtc.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación DTC

In [ ]:
scoresDTC = cross_val_score(resultDTC, train_demonicSalmon_x, train_demonicSalmon_y, cv=10)
print(scoresDTC)

Matriz de Confusión

In [ ]:
y_pred = dtc.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de DTC

In [ ]:
meanAcDTC = np.median(accuracyDTC)
stdAcDTC=np.std(accuracyDTC)
meanPrDTC = np.median(precisionDTC)
stdPrDTC=np.std(precisionDTC)
meanReDTC = np.median(recallDTC)
stdReDTC=np.std(recallDTC)
meanF1DTC = np.median(f1DTC)
stdF1DTC=np.std(f1DTC)
print(meanAcDTC, stdAcDTC)

Seleccionar caracteríticas RFC

In [ ]:
parametros = {'criterion':('gini','entropy','log_loss'),'bootstrap':[True, False]}
accuracyRFC = []
precisionRFC = []
recallRFC = []
f1RFC = []
rfc = RFC()
rfc = GridSearchCV(rfc, parametros)
for i in range(10):
  rfc.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultRFC = rfc.predict(test_demonicSalmon_x)
  accuracyRFC.append(accuracy_score(resultRFC, test_demonicSalmon_y))
  precisionRFC.append(precision_score(resultRFC, test_demonicSalmon_y, average=selectedAverage))
  recallRFC.append(recall_score(resultRFC, test_demonicSalmon_y, average=selectedAverage))
  f1RFC.append(f1_score(resultRFC, test_demonicSalmon_y, average=selectedAverage))

Entrenamiento RFC

In [ ]:
resultRFC = rfc.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación RFC

In [ ]:
scoresRFC = cross_val_score(resultRFC, test_demonicSalmon_x, test_demonicSalmon_y, cv=10)
print(scoresRFC)

Matriz de Confusión

In [ ]:
y_pred = rfc.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de RFC

In [ ]:
meanAcRFC = np.median(accuracyRFC)
stdAcRFC=np.std(accuracyRFC)
meanPrRFC = np.median(precisionRFC)
stdPrRFC=np.std(precisionRFC)
meanReRFC = np.median(recallRFC)
stdReRFC=np.std(recallRFC)
meanF1RFC = np.median(f1RFC)
stdF1RFC=np.std(f1RFC)

Selección de características AdaBoost

In [ ]:
parametros = {'n_estimators':[20, 100],'algorithm':('SAMME', 'SAMME.R'),'learning_rate':[1, 10]}
accuracyABC = []
precisionABC = []
recallABC = []
f1ABC = []
abc = ABC()
abc = GridSearchCV(abc, parametros)
for i in range(10):
  abc.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultABC = abc.predict(test_demonicSalmon_x)
  accuracyABC.append(accuracy_score(resultABC, test_demonicSalmon_y))
  precisionABC.append(precision_score(resultABC, test_demonicSalmon_y, average=selectedAverage))
  recallABC.append(recall_score(resultABC, test_demonicSalmon_y, average=selectedAverage))
  f1ABC.append(f1_score(resultABC, test_demonicSalmon_y, average=selectedAverage))

Entrenamiento AdaBoost

In [ ]:
resultABC = abc.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación ABC

In [ ]:
scoresABC = cross_val_score(resultABC, test_demonicSalmon_x, test_demonicSalmon_y, cv=10)
print(scoresABC)

Matriz de Confusión

In [ ]:
y_pred = abc.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de ABC

In [ ]:
meanAcABC = np.median(accuracyABC)
stdAcABC=np.std(accuracyABC)
meanPrABC = np.median(precisionABC)
stdPrABC=np.std(precisionABC)
meanReABC = np.median(recallABC)
stdReABC=np.std(recallABC)
meanF1ABC = np.median(f1ABC)
stdF1ABC=np.std(f1ABC)

Selección de parámetros NaiveBayes

In [ ]:
accuracyNB = []
precisionNB = []
recallNB = []
f1NB = []
nb = NB()
for i in range(10):
  nb.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultNB = nb.predict(test_demonicSalmon_x)
  accuracyNB.append(accuracy_score(resultKNN, test_demonicSalmon_y))
  precisionNB.append(precision_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))
  recallNB.append(recall_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))
  f1NB.append(f1_score(resultKNN, test_demonicSalmon_y, average=selectedAverage))

Entrenamiento GaussianNN

In [ ]:
resultNB = nb.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación NB

In [ ]:
scoresNB = cross_val_score(resultNB, test_demonicSalmon_x, test_demonicSalmon_y, cv=10)
print(scoresNB)

Matriz de confusión

In [ ]:
y_pred = nb.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de NB

In [ ]:
meanAcNB = np.median(accuracyNB)
stdAcNB=np.std(accuracyNB)
meanPrNB = np.median(precisionNB)
stdPrNB=np.std(precisionNB)
meanReNB = np.median(recallNB)
stdReNB=np.std(recallNB)
meanF1NB = np.median(f1NB)
stdF1NB=np.std(f1NB)

Entrenamiento Red Neuronal (MLP)

In [ ]:
mlp = MLP(hidden_layer_sizes=(150,100,50),max_iter=200,activation='relu',solver='adam')
accuracyMLP = []
precisionMLP = []
recallMLP = []
f1MLP = []
for i in range(10):
  mlp.fit(train_demonicSalmon_x, train_demonicSalmon_y)
  resultMLP = mlp.predict(test_demonicSalmon_x)
  accuracyMLP.append(accuracy_score(resultMLP, test_demonicSalmon_y))
  precisionMLP.append(precision_score(resultMLP, test_demonicSalmon_y, average=selectedAverage))
  recallMLP.append(recall_score(resultMLP, test_demonicSalmon_y, average=selectedAverage))
  f1MLP.append(f1_score(resultMLP, test_demonicSalmon_y, average=selectedAverage))

In [ ]:
resultMLP = mlp.fit(train_demonicSalmon_x, train_demonicSalmon_y)

Validación MLP

In [ ]:
scoresMLP = cross_val_score(resultMLP, test_demonicSalmon_x, test_demonicSalmon_y, cv=10)
print(scoresMLP)

Matriz de confusión MLP

In [ ]:
y_pred = mlp.predict(test_demonicSalmon_x)
cm = confusion_matrix(test_demonicSalmon_y, y_pred, labels=labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.show()

Média y varianza de MLP

In [ ]:
meanMLP=np.median(accuracyMLP)
stdMLP=np.std(accuracyMLP)
meanAcMLP = np.median(accuracyMLP)
stdAcMLP=np.std(accuracyMLP)
meanPrMLP = np.median(precisionMLP)
stdPrMLP=np.std(precisionMLP)
meanReMLP = np.median(recallMLP)
stdReMLP=np.std(recallMLP)
meanF1MLP = np.median(f1MLP)
stdF1MLP=np.std(f1MLP)
print(meanAcMLP, stdAcMLP)

Ejecutar el siguiente bloque para ver métricas finales

In [ ]:
print("Resultados KNN/ Accuracy: ",meanAcKNN, stdAcKNN, " Precision", meanPrKNN, stdPrKNN, " Recall:", meanReKNN, stdReKNN, " F1:",meanF1KNN, stdF1KNN)
print("Resultados SVM/ Accuracy: ",meanAcSVC, stdAcSVC," Precision",meanPrSVC, stdPrSVC," Recall:",meanReSVC, stdReSVC," F1:",meanF1SVC, stdF1SVC)
print("Resultados LSVM/ Accuracy: ",meanAcLinearSVM, stdAcLinearSVM," Precision",meanPrLinearSVM, stdPrLinearSVM," Recall:",meanReLinearSVM, stdReLinearSVM," F1:",meanF1LinearSVM, stdF1LinearSVM)
print("Resultados GPC/ Accuracy: ",meanAcGPC, stdAcGPC," Precision",meanPrGPC, stdPrGPC," Recall:",meanReGPC, stdReGPC," F1:",meanF1GPC, stdF1GPC)
print("Resultados DTC/ Accuracy: ",meanAcDTC, stdAcDTC," Precision",meanPrDTC, stdPrDTC," Recall:",meanReDTC, stdReDTC," F1:",meanF1DTC, stdF1DTC)
print("Resultados RFC/ Accuracy: ",meanAcRFC, stdAcRFC," Precision",meanPrRFC, stdPrRFC," Recall:",meanReRFC, stdReRFC," F1:",meanF1RFC, stdF1RFC)
print("Resultados AdaBoost/ Accuracy: :",meanAcABC, stdAcABC," Precision",meanPrABC, stdPrABC," Recall:",meanReABC, stdReABC," F1:",meanF1ABC, stdF1ABC)
print("Resultados GaussianNN/ Accuracy: ",meanAcNB, stdAcNB," Precision",meanPrNB, stdPrNB," Recall:",meanReNB, stdReNB," F1:",meanF1NB, stdF1NB)
print("Resultados MLP/ Accuracy: ",meanAcMLP, stdAcMLP," Precision",meanPrMLP, stdPrMLP," Recall:",meanReMLP, stdReMLP," F1:",meanF1MLP, stdF1MLP)